In [1]:
from sklearn.tree import DecisionTreeClassifier

import os
os.chdir('/Users/kshitijphulare/DLH-Proj/')
from pre_processing import *
from main import *
# from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import KFold
import csv
import pandas as pd

In [2]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous_Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]

with open("./performance_DT.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [5]:
total_macro=[]
total_micro = []

In [6]:
for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = tf_idf(train_preprocessed_df, morbidity)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    total_macro.append(f1_macro)
    total_micro.append(f1_micro)
    with open("./performance_DT.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./performance_DT.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ['Average']
    row.extend([sum(total_macro)/len(total_macro),sum(total_micro)/len(total_micro)])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
(514, 600) (514,) (58, 600) (58,)
(514, 600) (514,) (58, 600) (58,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
(515, 600) (515,) (57, 600) (57,)
Macro F1 score: 0.9078886825318037 and Micro F1 Score 0.9632183908045976
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(493, 600) (493,) (55, 600) (55,)
(494, 600) (494,) (54, 600) (54,)
(494, 600) (494,) (54, 600) (54,)
Macro F1 score: 0.7906006954644307 and Micro F1 Score 0.7972053872053871
CHF
(243, 600) (243,) Counter({1.0: 243})
(218, 600) (218,) (25, 600